# Загрузка библиотек

In [1]:
!pip install hiclass

# MLFLOW

In [33]:
import pandas as pd
import mlflow
from sklearn.model_selection import train_test_split, ParameterGrid
from hiclass import LocalClassifierPerNode
from sklearn.ensemble import RandomForestClassifier
import torch
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from tqdm.notebook import tqdm

models =  [["CatBoost", CatBoostClassifier]]

## Создание датасетов

In [34]:

train = pd.read_csv("datasets/sbert_large_mt_nlu_ru_train_mean.csv")
test = pd.read_csv("datasets/sbert_large_mt_nlu_ru_test_mean.csv")

In [35]:
train[["Группа тем", "Тема"]]

,Группа тем,Тема
0,Мусор/Свалки/ТКО,★ Уборка/Вывоз мусора
1,Социальное обслуживание и защита,Дети и многодетные семьи
2,ЖКХ,Жалобы на управляющие компании
3,Социальное обслуживание и защита,Аварийное жилье/переселение
4,Общественный транспорт,Содержание остановок
...,...,...
15090,Физическая культура и спорт,Строительство спортивной инфраструктуры
15091,Коронавирус,Порядок и пункты вакцинации
15092,Здравоохранение/Медицина,★ Оказание медицинской помощи не в полном объе...
15093,Дороги,Необходима установка и замена дорожных ограждений


rf = RandomForestClassifier()
classifier = LocalClassifierPerNode(local_classifier=rf)

In [47]:
param_grid = {
    'iterations': [100,200,300,400]
}
grid = list(ParameterGrid(param_grid))

In [48]:
mlflow.set_experiment('hiclass_classifiers')

<Experiment: artifact_location='file:///workspace/mlruns/109313160040224942', creation_time=1700831294812, experiment_id='109313160040224942', last_update_time=1700831294812, lifecycle_stage='active', name='hiclass_classifiers', tags={}>

In [49]:
for model_ in models:
    model_name, classifier = model_
    

    train_, test_ = train, test
    print(train_.drop(columns=["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]).values)
    print( train_[["Группа тем", "Тема"]].values)
    for params in grid:
        # try:
        with mlflow.start_run(nested=True):
            classifier_model = LocalClassifierPerNode(local_classifier=classifier(**params, verbose=0, random_seed=42))
            classifier_model.fit(train_.drop(columns=["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]), 
                                 train_[["Группа тем", "Тема"]])

            predictions = classifier_model.predict(test_.drop(columns=["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]))
            print(predictions.shape)
            accuracy = accuracy_score(test_["Группа тем"], predictions[:,0])
            f1 = f1_score(test_["Группа тем"], predictions[:,0], average='weighted')
            report = classification_report(test_["Группа тем"], predictions[:,0], output_dict=True)
            report_text =  classification_report(test_["Группа тем"], predictions[:,0])


            mlflow.log_metric("report_accuracy", report['accuracy'])
            mlflow.log_metric("macro avg_precision", report['macro avg']['precision'])
            mlflow.log_metric("macro avg_recall", report['macro avg']['recall'])
            mlflow.log_metric("macro avg_f1-score", report['macro avg']['f1-score'])
            mlflow.log_metric("weighted avg_precision", report['weighted avg']['precision'])
            mlflow.log_metric("weighted avg_recall", report['weighted avg']['recall'])
            mlflow.log_metric("weighted avg_f1-score", report['weighted avg']['f1-score'])

            mlflow.log_text(report_text, "classification_report.txt")

            mlflow.set_tag("model_name", model_name)
            mlflow.set_tag("dataset_name", "sbert_large_mt_nlu_ru_train_mean")
            mlflow.set_tag("model_name", "catboost")
            mlflow.log_params(params)
            mlflow.log_metrics({'accuracy': accuracy, 'f1-weighted': f1})
        # except Exception as e:
        #     error_name = type(e).__name__
        #     print(f"Caught an error: {error_name}")

[[ 8.22169423e-01  3.29378903e-01 -1.76240861e-01 ...  4.32861954e-01
   6.24679364e-02  9.37354863e-01]
 [ 3.03478539e-01  8.19452107e-02 -6.36221409e-01 ... -1.60302725e-02
  -6.20140791e-01  9.36790466e-01]
 [-7.41726831e-02 -1.86198339e-01  7.69123912e-01 ...  3.94569516e-01
  -7.67487735e-02  4.94751960e-01]
 ...
 [-8.86966228e-01  1.61663871e-02 -1.49601591e+00 ...  1.53779492e-01
  -5.94152957e-02  1.04251623e+00]
 [ 8.36172700e-01 -1.24470942e-01 -1.37997425e+00 ...  2.24936366e-01
   2.92432815e-01  6.49066389e-01]
 [ 8.10131431e-01  1.12581137e-03 -1.11698651e+00 ...  3.20196748e-01
  -5.11597991e-01  4.01751906e-01]]
[['Мусор/Свалки/ТКО' '★ Уборка/Вывоз мусора']
 ['Социальное обслуживание и защита' 'Дети и многодетные семьи']
 ['ЖКХ' 'Жалобы на управляющие компании']
 ...
 ['Здравоохранение/Медицина'
  '★ Оказание медицинской помощи не в полном объеме/отказ в оказании медицинской помощи']
 ['Дороги' 'Необходима установка и замена дорожных ограждений']
 ['ЖКХ' 'Отсутствие гор

KeyboardInterrupt: 

In [32]:
predictions[:,0]

array(['Социальное обслуживание и защита', 'ЖКХ',
       'Социальное обслуживание и защита', ...,
       'Социальное обслуживание и защита', 'Дороги', 'Благоустройство'],
      dtype='<U162')

In [41]:
print(classification_report(test_["Группа тем"], predictions[:,0]))

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                          precision    recall  f1-score   support

                            Безопасность       0.81      0.47      0.59        94
                         Благоустройство       0.51      0.50      0.50       806
                           Газ и топливо       0.50      0.04      0.07        26
           Государственная собственность       0.00      0.00      0.00         7
                                  Дороги       0.64      0.77      0.70      1005
                                     ЖКХ       0.72      0.72      0.72       870
                Здравоохранение/Медицина       0.79      0.90      0.84      1516
                             Коронавирус       0.67      0.33      0.44       264
                                Культура       0.33      0.04      0.08        23
                     МФЦ "Мои документы"       0.00      0.00      0.00         6
                             Мобилизация       1.00      0.12      0.21        43
               

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
